In [123]:
import math

def f1(x1, x2):
  return (x1-x2)**2 + 1/9*(x1 + x2 - 10)**2

def f2(x1, x2):
  return 100*(x2 - x1**2)**2 + (1-x1)**2

def f3(x1, x2):
  return (math.sin(x1-0.6) - x2 - 1.6)**2 + (3*x1 - math.cos(x2) - 0.9)**2


In [124]:
import numpy as np

def hessian(func, h, *args):
  n = len(args)
  initial = np.array(args)
  hessian = np.zeros((n, n))
  for i in range(n):
    for j in range(n):
      ei = np.zeros(n)
      ei[i] = 1
      ej = np.zeros(n)
      ej[j] = 1
      f1 = func(*(initial + h * ei + h * ej))
      f2 = func(*(initial + h * ei - h * ej))
      f3 = func(*(initial - h * ei + h * ej))
      f4 = func(*(initial - h * ei - h * ej))
      numdiff = (f1-f2-f3+f4)/(4*h*h)
      hessian[i, j] = numdiff
  return hessian


In [125]:
import pandas as pd
diffs = [10 ** i for i in range(-2, -10, -1)]
data = [[f1, 0, 1], [f2, -1.2, 1], [f3, 0, 0]]
df = pd.DataFrame([[hessian(func, h, *args)
                  for h in diffs] for [func, *args] in data])
pd.options.display.float_format = '{:,}'.format


def flatten_columns(df, cols):
    """Flattens multiple columns in a data frame"""
    flattened_cols = {}
    for col in cols:
        flattened_cols[col] = pd.DataFrame(
            [value for (index, values) in df[col].items() for value in values])
    flattened_df = pd.concat([flattened_cols[col] for col in cols], axis=1)
    return flattened_df


df = flatten_columns(df, df.columns)
df.index = pd.Index([y for x in ['F1', 'F2', 'F3'] for y in [x] * 2])
df.columns = pd.Index([d for diff in diffs for d in [diff, diff]], name='h')
df.transpose()

,F1,F1,F2,F2,F3,F3
h,,,,,,
0.01,2.2222222222234578,-1.7777777777761017,"1,330.0799999999756",479.99999999998266,16.91782319823698,-1.6506437187624456
0.01,-1.7777777777761017,2.2222222222234578,479.99999999998266,199.99999999999574,-1.6506437187624456,-1.7996733483460758
0.001,2.2222222217926912,-1.7777777769900638,"1,330.0007999959007",479.99999999959186,16.917859230858312,-1.6506709545893727
0.001,-1.7777777769900638,2.2222222226808697,479.99999999959186,200.00000000219842,-1.6506709545893727,-1.7999967325543764
0.0001,2.2222222728629504,-1.7777777916450077,"1,330.0000072824503",480.0000000138027,16.917859646525812,-1.6506712441355376
0.0001,-1.7777777916450077,2.2222222728629504,480.0000000138027,199.9999998503199,-1.6506712441355376,-1.7999999268880629
1e-05,2.222213524305516,-1.7777868066559674,"1,329.9999590543619",480.00000418824123,16.917862666332436,-1.6506707112284855
1e-05,-1.7777868066559674,2.222213524305516,480.00000418824123,199.9999454938006,-1.6506707112284855,-1.799995708040569
1e-06,2.2222224060897133,-1.7776891070298007,"1,330.0009982231131",480.0018160722175,16.918022538447985,-1.6506795930126827


In [126]:
df = df.transpose()
actual_h = pd.DataFrame([[20 / 9, -16 / 9, 1330, 480, 16.9178596, -1.65067],
                        [-16 / 9, 20 / 9, 480, 200, -1.65067, -1.8]] * len(diffs), columns=df.columns)
actual_h.index = df.index
df.subtract(actual_h).abs()


,F1,F1,F2,F2,F3,F3
h,,,,,,
0.01,1.2354561818028742e-12,1.6759926779741363e-12,0.07999999997559826,1.7337242752546445e-11,3.6401763019000555e-05,2.6281237554437453e-05
0.01,1.6759926779741363e-12,1.2354561818028742e-12,1.7337242752546445e-11,4.263256414560601e-12,2.6281237554437453e-05,0.0003266516539242037
0.001,4.2953107737275786e-10,7.87713894112585e-10,0.0007999959007065627,4.0813574742060155e-10,3.691416878837117e-07,9.545893726592425e-07
0.001,7.87713894112585e-10,4.5864734232736737e-10,4.0813574742060155e-10,2.19841922444175e-09,9.545893726592425e-07,3.267445623622578e-06
0.0001,5.064072805538444e-08,1.3867230030939481e-08,7.282450269485707e-06,1.3802718967781402e-08,4.65258125359469e-08,1.2441355374814833e-06
0.0001,1.3867230030939481e-08,5.064072805538444e-08,1.3802718967781402e-08,1.4968009054427966e-07,1.2441355374814833e-06,7.311193717107756e-08
1e-05,8.697916706434938e-06,9.028878189765166e-06,4.094563814760477e-05,4.188241234714951e-06,3.066332435963659e-06,7.112284854393636e-07
1e-05,9.028878189765166e-06,8.697916706434938e-06,4.188241234714951e-06,5.450619940461365e-05,7.112284854393636e-07,4.291959430968717e-06
1e-06,1.8386749101040323e-07,8.867074797702656e-05,0.0009982231131289154,0.0018160722174798138,0.00016293844798553891,9.59301268266266e-06
